# Notes about this lecture
Because of the Corona virus outbreak, this lecture will not be held in the classroom but online only. Further, the lecture will only be available in this written form. In order to offer support for the students we will use the gitlab issue tracker as a question & answer forum: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and individual videoconference sessions when needed.

## Software

### Necessary software
Please install the following tools:
* python3 (https://www.python.org/downloads/ version 3.8.2 is fine.
Python is a prerequisite for jupyter)
* jupyter-notebook (https://jupyter.org/install.html)
* **Hint for Windows and OSX**: Try to install conda or miniconda (https://docs.conda.io/en/latest/miniconda.html) first. This will install Python and jupyter-notebook automatically.

### Optional (but highly recommended) software
* git (https://git-scm.com/download/). Git is harder to install but not strictly necessary. **Hint**: On Windows Git will automatically install a Linux compatible shell which can then be found as 'Git BASH'.
* If git is not available, solutions shall be uploaded on https://polybox.ethz.ch instead and the folder shall be shared with the lecturers. 

## Support
**For any issues please use the forum** at: https://git.ee.ethz.ch/python-for-engineers/class-fs20-forum and follow the instructions therein. In case of need, we will open a room using Jitsi or BigBlueButton and share the audio, video or the screen: make sure you have a microphone and speakers functioning. 

This service is offered only **during the normal lecture hours**.

# Obtaining the material for this lecture
### If git is available on your system (preferred option)
Pull the new material from the upstream repository:

```bash
cd class-fs20
git pull upstream master
```

Then launch the jupyter-notebook and open the Lecture_XX file:

```bash
anaconda # Only on ETH computers to load the Python environment.
jupyter-notebook &
```

### If git is **not** available on your system
Download the latest material from:
https://git.ee.ethz.ch/python-for-engineers/class-fs20/-/archive/master/class-fs20-master.zip
and unpack it on your computer.

# Refreshing previous lecture

Open the pdf of the past lecture (licences) and quickly read through it. This will help fixing the learned notions into the long-term memory.

### ✏️ $\mu$-exercise

After having refreshed the past lecture, switch to the Exercise notebook and complete $\mu$-exercise **0**.

# Known issues

*Read this if the code examples fail to run with an error like `'await' outside function`*.

This lecture requires an up-to-date version of Jupyter notebook. Likely, this is already installed. However, if the code examples cannot be executed then a more recent version of Jupyter notebook should be installed.

## Installing a more recent version of Jupyter notebook
This is only necessary if the above error occurs.

```sh
pip3 install --user --upgrade jupyterlab
```

Alternatively, if the above command does not help it is possible to create a fresh installation of Jupyter notebook in a new *virtual environment*:
```sh
# On Linux/Debian install python3-venv first.
python3 -m venv myenv # Create a new virtual environment.
source myenv/bin/activate # Activate the virtual environment.
pip3 install jupyterlab
jupyter-notebook
```

# `async/await` - Asynchronous I/O with `asyncio`

## Introduction - Non-preemptive multitasking

This lecture introduces *multitasking* in Python with the `async/await` syntax. Generally, multitasking is understood as the concurrent execution of tasks or programs. This means that multiple programms or functions can be running at the same time. Tasks can run truely parallel on multiple CPU cores or they can also share computation time on a single core. If multiple tasks run on the same CPU core then the CPU time must be split among the tasks. This means that tasks must sometimes be interrupted to let other tasks use the CPU. Multitasking can be roughly split into two classes depending on how tasks can be interrupted, *preemptive* and *non-preemtive* multitasking:

1) *Preemptive* means that a task can be paused by the operating system for a *context switch*, i.e. to free the CPU for another task to run. Tasks themselves have little control over where in their code they can be paused. Preemptive multitasking is often implemented with *threads*.

2) On the other hand in *non-preemptive* multitasking tasks explicitly tell when they want to be paused. For this reason non-preemptive multitasking is often called *cooperative* multitasking. Further, names like 'coroutines' or 'fibers' can be encountered.
Cooperative multitasking usually has less overhead than operating-system threads. Context switches are faster for cooperative multithreading and tasks have less memory overhead. This makes cooperative multitasking very well suited for highly concurrent and *I/O-bound* applications such as networking where thousands of connections must be handled at the same time. *I/O-bound* means that a program is often busy waiting for data to arrive or to be sent while the CPU is idle and could be used to work on another task.

Quoting Wikipedia: "*In computing, preemption is the act of temporarily interrupting a task being carried out by a computer system, without requiring its cooperation, and with the intention of resuming the task at a later time.*" https://en.wikipedia.org/wiki/Preemption_(computing)

## `async/await` Syntax
With version 3.5 Python introduced the `async/await` syntax to write *coroutines*. Coroutines can be thought of as functions that can be suspended and resumed. Therefore their execution is not necessarily synchronous but *asynchronous* and hence they are defined with `async def`. This means once an `async` function or *coroutine* does not return immediately but somewhen later such that other code can be run between the start and end of the coroutine. Notice that coroutines behave similarly to *generators*. The `await` keyword is used to explicitly  wait for completion of a coroutine.

The following shows a simple example of the `async/await` syntax:

In [ ]:
async def asyncFunction():
    print("Hello!")
    return 7

# Instead of returning 7 this creates a 'coroutine'.
task = asyncFunction()
# Notice that `task` is not 7 but a 'coroutine'.
print(type(task))

In [ ]:
# Wait for the coroutine to complete and get the return value.
# Notice that the above cell must be run before.
result = await task
print("result =", result)

Let's have a look at the asynchronous `sleep` function in the `asyncio` library. `sleep` is an `async` function which waits for a given amount of time until it completes. It will be used in the next examples.

In [ ]:
import asyncio

print("Going to sleep...")
await asyncio.sleep(2)
print("Awake again!")

`async` functions can call and await other `async` functions as shown here:

In [ ]:
import asyncio

async def double_sleep(sleep_time):
    # This `async` function calls other `async` functions.
    print("Sleep for a moment.")
    await asyncio.sleep(sleep_time)
    print("Sleep for another moment.")
    await asyncio.sleep(sleep_time)
    print("Done!")
    
await double_sleep(1)

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 1!

## Running tasks in parallel: `asyncio.create_task`

The following example defines two `async` functions, each of which takes some time to complete. Internally they call `asyncio.sleep` for creating the delay. Here the two coroutines are awaited one after the other. This means that the second coroutine will not start execution before the first finished and hence there is no parallel execution yet.

In [ ]:
import asyncio
import time

async def asyncFunction1():
    print("asyncFunction1() start")
    # Sleep for 1 second.
    # Here this coroutine is suspended for 1 second an another can be run.
    await asyncio.sleep(1)
    print("asyncFunction1() end")
    return 1

async def asyncFunction2():
    print("asyncFunction2() start")
    # Sleep for 2 seconds.
    await asyncio.sleep(2)
    print("asyncFunction2() end")
    return 2
    
    
start_time = time.time()
# Create and wait for the coroutines one after the other.
return_value1 = await asyncFunction1()
return_value2 = await asyncFunction2()
end_time = time.time()

assert return_value1 == 1
assert return_value2 == 2

# Notice that it takes 3 seconds to complete both tasks.
# There seems to be no execution in parallel.
print("Duration to complete all tasks: {:0.3f}s".format(end_time-start_time))

In cases like this, parallel execution is very desirable. Both coroutines spend a long time just waiting and not occupying the CPU. Therefore, they can be scheduled in parallel as shown in the following example:

In [ ]:
# Schedule new tasks for parallel execution.
# Therefore it should take less time to complete them compared to
# sequential execution.
# The `create_task` function wraps a coroutine in a parallel task.
task1 = asyncio.create_task(asyncFunction1())
task2 = asyncio.create_task(asyncFunction2())

# Notice that the type of the tasks is `Task`, not `coroutine`.
print(type(task1))

# Now the two tasks are executed independently.

start_time = time.time()
# Wait for the tasks to complete.
return_value1 = await task1
return_value2 = await task2
end_time = time.time()

assert return_value1 == 1
assert return_value2 == 2

# Notice that it takes only 2 seconds to complete both tasks
# instead of 2+1 seconds.
print("Duration to complete all tasks: {:0.3f}s".format(end_time-start_time))

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 2!

## Event loops

https://docs.python.org/3/library/asyncio-eventloop.html

"The event loop is the core of every asyncio application. Event loops run asynchronous tasks and callbacks, perform network IO operations, and run subprocesses."

In other words, an *event loop* is a program that manages the scheduling of tasks and coroutines. When a coroutine is paused because it calls `await` then the control goes back to the event loop which will decide which task to execute next. As the name suggests the *event loop* handles *events* such as an elapsed timer or arriving network packets. Upon such an event the *event loop* resumes tasks that are waiting for this event. The `asyncio` library implements an event loop. There are also other event loop implementations around such as [`uvloop`](https://uvloop.readthedocs.io/).

The event loop usually must be started explicitly but here **in the Jupyter notebook there is already an event loop running**. Therefore, we can directly use `await` in the code.

In a separate Python script, where no event loop is running by default, `async` functions must be run differently. The `await` keyword cannot be used in non-async functions, but only inside `async` functions. The highest-level `async` function can, for instance, be started with `asyncio.run` which creates a new event loop. An example of this is given below.

```python
# This code must be in a separate Python script file, not in Jupyter notebook.
import asyncio

async def main():
    print("Wait...")
    await asyncio.sleep(1)
    print("Done!")
   
# Run a coroutine if there is no running event loop yet.
asyncio.run(main())
```

### Accessing the current event loop
The current event loop can be accessed as follows. This will be used in following sections.

In [ ]:
# There is already a running event loop in the Jupyter notebook.
asyncio.get_running_loop()

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 3.

### Futures
The coroutines and tasks introduced above are *awaitable* which means they can be used in an `await` expression. There is a third awaitable type that is often used which is called `Future`. A *Future* object, or in short a *Future*, represents a future result of an asynchronous operation which will arrive somewhen in the future. With the `await` keyword it is possible to wait for the data of the `Future` to arrive.

More specifically, when used in an `await` expression a `Future` behaves as follows: `await` only waits until the future value is *set* by the `.set_result()` or by the `set_exception()` methods. For more methods see: https://docs.python.org/3/library/asyncio-future.html .

The example below illustrates how a `Future` object can be created and used:

In [ ]:
import asyncio

# Create a `Future`.
# Notice that the future is created by the current event loop.
loop = asyncio.get_running_loop()
future = loop.create_future()
print("Type of the future is", type(future))

# Now the future does not yet hold a value.
print("Is the future already done?", future.done())

# Create a task that will somewhen set the futures value.
async def setMyFuture(future): # Pass the future.
    await asyncio.sleep(2)
    future.set_result(42) # Set the value of the future.
    await asyncio.sleep(4)

# Start the tasks that will somewhen set our future.
# `create_task` makes sure that `setMyFuture` is run in parallel to the code here.
task = asyncio.create_task(setMyFuture(future))

# Wait for the future to get assigned its value.
print("Wait for future...")
future_value = await future # This waits until `.set_result()` is called on the future.
print("Value of future: ", future_value)

# Now the future should be 'done'.
print("Is the future already done?", future.done())

# Wait for the parallel task to complete (extra 4 seconds).
await task 
print("Task completed.")

## Running tasks in a separate thread

Non-preemptive tasks (i.e. tasks which explicitly tell when they want to be paused) can *block* other tasks from being executed. This is especially problematic when tasks perform a long computation or blocking operation such as accessing the disk. To protect other tasks from being blocked, long computations can be executed in another operating-system thread. This also gives the operating system the opportunity to run the *heavy load* on another CPU core if multiple cores are available.

To execute tasks in different threads the `ThreadPoolExecutor` can be used in combination with the `run_in_executor()` method. The class `ThreadPoolExecutor` is a subclass of the abstract `Executor` class that provides methods to execute calls asynchronously. This allows to create a *pool* of different threads. The `run_in_executor()` function can now be used to execute functions in threads of the created *pool*.

An example of two task running in different threads is given below.

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import time

# Create an 'executor' to run selected tasks on separate threads.
thread_pool = ThreadPoolExecutor()

def heavy_computation(name: str, parameter): # This is a normal function, there is no `async`.
    print("Starting heavy computation '{}'".format(name))
    # In contrast to `asyncio.sleep` `time.sleep` would block other coroutines.
    time.sleep(3) # This is a placeholder for a "heavy" computation that takes 3 seconds.
    return "This is the result of '{}': {}".format(name, parameter)

# Get the current event loop.
loop = asyncio.get_event_loop()

# Schedule two tasks to be run in parallel on separate threads.
task1 = loop.run_in_executor(thread_pool, # Pass the 'Executor'.
                             heavy_computation, # Pass the function to be executed.
                             "task1",
                             "We want this to be printed first." # Parameters can be passed to the function.
                            )
task2 = loop.run_in_executor(thread_pool,
                             heavy_computation,
                             "task2",
                             "We want this to be printed second."
                            )

# Notice that here the two tasks have been started already.
# Because they were started in separate threads it take almost no time
# to get here.

# Start measuring the time it takes for the following tasks to complete.
start_time = time.time()

# Imagine we now need the result of task1.
# Therefore we have to wait for task1 to be completed.
result1 = await task1
print(result1)
# Wait for task2 to be completed.
result2 = await task2
print(result2)
end_time = time.time()

# Notice that it takes only 3 seconds to complete both tasks instead of 2*3 seconds.
print("Total duration: {:.2f}s".format(end_time-start_time))


### ✏️ $\mu$-exercise
Solve $\mu$-exercise 4!

## Communication between tasks

Often tasks are not independent of each other but require to exchange information or get synchronized. The `asyncio` library offers the following primitives for communication and synchronization:

* `asyncio.Queue` for passing messages between tasks.
* `asyncio.Event` for synchronization.
* `asyncio.Lock` & `asyncio.Semaphore` for granting exclusive access to a code region.

Synchronization primitives will only be covered briefly in an optional section. The interested reader shall consult the official documentation: https://docs.python.org/3/library/asyncio-sync.html

The following example makes use of `asyncio.Queue` for sending messages to a concurrent task. `Queue` is a *first-in-first-out* (FIFO) data structure. This means that the first object put into the queue is the first to be taken out again. Here only two functions of the `Queue` class are used: `put` and `get`. `get` can be used to fetch an object from the queue. If there is nothing in the queue, `get` will wait until there is something. `put` writes objects into the queue. Queues can have a limited size. If a queue is full, then `put` will wait until `get` creates a free space in the queue.

In [ ]:
import asyncio


async def consumer(queue: asyncio.Queue):
    """
    'comsumes' messages from the queue.
    Read and print messages from the queue in a loop.
    Make a 3 seconds break in between messages.
    """
    while True:
        # Wait for the next message to come from the queue.
        msg = await queue.get()
        print("Got a message from the queue:", msg)
        
        if msg is None:
            # Break the loop if `None` arrives.
            print("Stopping consumer task.")
            break
        # Sleep between messages.
        await asyncio.sleep(3)

# Create a queue object.  
# Queues can optionally be limited to a maximal size.
# Change the maximal queue size and observe what happens!
queue = asyncio.Queue(maxsize = 1)

# Start the consumer task.
consumer_task = asyncio.create_task(consumer(queue))

# Put some messages in the queue.
await queue.put("message 1")
print("Message is in the queue now.")

await queue.put(["message", "2", "is", "a", "list"]) # Messages can be any object.
print("Message is in the queue now.")

await queue.put(None) # This will stop the consumer task.
print("Message is in the queue now.")

await consumer_task
print("Done")

### ✏️ $\mu$-exercise
Solve $\mu$-exercise 5!

## Synchronization with `Event` (optional)

Read the documentation on the synchronization primitives: https://docs.python.org/3/library/asyncio-sync.html

The following gives a quick example on how to use `asyncio.Event` for synchronization.

In [ ]:
import asyncio

async def wait_for_event(event: asyncio.Event):
    print("Waiting for event...")
    await event.wait() # This will wait until `event.set()` is called.
    print("... event arrived!")

# Create an event object.
event = asyncio.Event()

# Start concurrent task that is waiting for the event.
task = asyncio.create_task(wait_for_event(event))

await asyncio.sleep(3)

# Trigger the event.
print("Trigger the event.")
event.set()

await task

## Synchronization with `Lock` (optional)

In some cases there are code sections that should not be executed by more than one task at the same time. Imagine there are multiple tasks trying to print something to the standard output. In this case it could be possible that the printed output of the tasks shows interleaved characters or interleaved lines. The following code shows a simple example for this: `slow_print` takes a list of strings and prints each of the strings. After printing a line it sleeps for a moment to make the effect more pronounced. Here it is possible that `slow_print` is run by two tasks at the same time. Because `slow_print` uses `await` in the printing loop it is also possible that the printing loop is interrupted and another task is run in the meantime. This can cause the lines of two tasks to be mixed.

In [ ]:
import asyncio

async def slow_print(lines: list):
        print("Starting slow_print...")
        for line in lines:
            print(line)
            # While waiting here another task can start printing.
            await asyncio.sleep(1)
        
task1 = asyncio.create_task(slow_print(["|", "|", "|"]))
task2 = asyncio.create_task(slow_print(["-", "-", "-", "-", "-"]))

await task1
print("task1 is done!")
await task2
print("task2 is done!")

# Notice that the output lines of the two tasks are interleaved.

To make sure the printing loop is executed by at most one task at a time `asyncio.Lock` can be used together with an `async with` block as shown in the code below. In other languages the concept of `Lock` is often called *mutex* ('mutual exclusion').

In [ ]:
import asyncio

# Create a lock object.
print_lock = asyncio.Lock()

async def slow_print(lines: list):
    # Make sure that the following block is never executed by more than one
    # task at the same time.
    print("Starting slow_print...")
    
    async with print_lock:
        print("Enter critical section.")
        for line in lines:
            print(line)
            await asyncio.sleep(1)
        print("Exit critical section.")
        
task1 = asyncio.create_task(slow_print(["|", "|", "|"]))
task2 = asyncio.create_task(slow_print(["-", "-", "-", "-", "-"]))

await task1
print("task1 is done!")
await task2
print("task2 is done!")

# Notice that now the lines are not interleaved anymore.

# Networking basics: IP, TCP & UDP
When sending and receiving data through the internet or local networks data is packaged into *IP* (*Internet Protocol*) packets. IP packets bundle a data fragment together with a source and destination address to tell the network where to bring the packet. From an application viewpoint IP packets are rarely used directly, instead further layers of abstraction are used. In practice this is very often *TCP* (*Transmission Control Protocol*) or *UDP* (*User Datagram Protocol*). Both of the protocols use IP packets but they add something on top.

UDP adds very little on top such as port numbers and simple checksums to detect transmission errors. Port numbers allow to create multiple endpoints on a single IP address. UDP is still *datagram* oriented like IP. This means that data transmission  happens on a *best-effort* way. UDP packets can be dropped on the network, reordered or duplicated. All this cases must be handled by the application.

In contrast TCP is *stream oriented*. TCP allows the application to send and receive streams of bytes without worrying whether some packets might get lost, reordered or duplicated. Also TCP detects if a connection is interrupted.

# Networking with sockets (without `asyncio`)

This section shows how data can be sent over the network the *old* way without `asyncio`. The concept used to manage network connections is called *socket* as in many other programming languages. Usually there is a distinction between *server* sockets and *client* sockets. A server socket waits for incoming connections and does not initiate connections. A client socket initiates the connection to a server.

In the following example we open a connection to the `ethz.ch` server on port 80. Once the connection is open a byte string is sent and the response byte string is received and printed.

In [ ]:
import socket

# Create a client socket and connect to the server.
# AF = Address Family
# INET = IPv4
# SOCK_STREAM = TCP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("ethz.ch", 80))

# Send a 'HTTP GET' request to the server.
# Understanding the HTTP protocol is not necessary for this lecture.
message_to_send = "GET /index.html HTTP/1.1\r\nHost: ethz.ch\r\n\r\n"
# Encode the message as bytes.
message_bytes = message_to_send.encode('utf-8')
s.sendall(message_bytes) # Send the bytes.

# Receive at most 4096 bytes of the response.
response = s.recv(4096)

print("Server response:")
# The expected response is a HTTP header telling
# us that the page has moved to https://ethz.ch/index.html.
print(response.decode('utf-8'))

# The response should be a HTTP header.

## Simple TCP Server

Open the file `00_tcp_socket_server.py` from the lecture folder. It contains a minimal example of a TCP server that waits for incoming connections on port 12345. Once a connection is opened it will send a message to the client and accept a message from the client and print it.

Launch the server script from the command line. It will not terminate but stay in an endless loop. (Terminate with Ctrl-c).

In case an error `OSError: [Errno 98] Address already in use` occurs this means that there is already an open server socket on port 12345. In this case make sure that scripts executed before are closed. Alternatively, it is also possible to change the port number for both the server and the client.

The following code now connects to the server on port 12345, prints the message from the server and sends a message to the server. Also observe the output of the server script on the terminal.

In [ ]:
# Client code that will connect to the server.

import socket
# Create a client socket and connect to the server.
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("localhost", 12345))

# Send data to the server.
s.sendall(b"Hi, I'm the client!\n")

# Receive at most 4096 bytes from the server.
input_data = s.recv(4096)

print("Data from server:")
print(input_data)

s.close()

# Notice that a `ConnectionRefusedError` can mean that the server is not running.

Linux users are also encouraged to use the `netcat` tool to connect to the running server. `netcat` is available on most Linux distributions, but possibly has to be installed first with the package manager.

`netcat` allows to send data from the standard input (e.g. user input on the terminal) over a TCP connection. Incoming data will be printed to the standard output.

```sh
# Connect to the server on 'localhost' with port 12345.
nc localhost 12345
```

Netcat can also be used as a server:
```sh
nc -l -p 12345 # Listen on Port 12345
```

### Limitations of sockets

The simple usage of a server socket is lacking the capability to handle multiple connections at the same time. The server example from above, as well as the example below do not accept a new connection as long there is an existing one. The `while True` loop is only started from the beginning once the old connection is finished.

```python
# Open a server socket.
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serversocket.bind(('localhost', 12345))
serversocket.listen()

while True:
    # Wait for a connection
    (sock, address) = serversocket.accept()
    # Do something with the connection.
    # In the mean time no new connections will be accepted.
```

The usual workaround is to create a new thread for every accepted connection. This is not covered in this lecture since `asyncio` provides a more convenient way to handle concurrent network connections.

## Networking with `asyncio`

Networking and cooperative multi-tasking can be very well combined. Usually when transmitting or receiving data there is a considerable amount of time spent just for waiting for data to arrive. In the meantime other connections could be handled. Further, networking is often highly concurrent especially on the server side. Therefore, the low overhead of tasks makes cooperative mult-tasking even more efficient than thread based concurrency.


In the following example a TCP connection is created to a server using the `asyncio` library. More precisely this code uses the *stream* API (https://docs.python.org/3/library/asyncio-stream.html) which allows to write and read data conveniently.

In [ ]:
# Connect to a server, read and write some data.
# Make sure the TCP server from above is still running!

import asyncio

# Open a TCP connection to a server.
# On success this creates a reader and writer stream.
reader, writer = await asyncio.open_connection('localhost', 12345)

# `reader` (StreamReader) can be used to receive data from the TCP connection.
# `writer` (StreamWriter) can be used to send data over the TCP connection.

message = "Hello, I'm the client using asyncio!"
# Send the message to the server.
writer.write(message.encode())
# Wait for the data to be sent.
await writer.drain()

# Wait for a piece of data coming in (at most 100 bytes).
line = await reader.read(100)
print(f'Received: {line.decode()}')

print('Close the connection')
writer.close()
# Wait for the writer to be properly closed.
await writer.wait_closed()

    

In the client code above there is no concurrency. Yet concurrent applications profit the most from `asyncio`. Concurrency is naturally present on the server side because the server should be able to handle multiple client connections simultaneously. The code below shows how a TCP server can be built using the `asyncio` library. Notice that there are two essential parts: The `main` coroutine which starts the server and the `handle_connection` coroutine which deals with the incoming connections. Once a connection is opened a new instance of the `handle_connection` coroutine will be created and run. This allows to run multiple connections at the same time.

```python
import asyncio

async def handle_connection(reader: asyncio.StreamReader, writer: asyncio.StreamWriter):
    # This is the coroutine that will be run for every incoming connection.
    
    # Do something with the reader and writer stream.
    data = await reader.read(100)
    writer.write(data)
    
    # ...
    
    await writer.drain()

    # Close the connection.
    writer.close()

async def main():
    # Start a TCP server.
    server = await asyncio.start_server(
        handle_connection, # This coroutine will be run for all incoming connections.
        # Tell from which addresses to accept connections. Put '0.0.0.0' to accept connections from all addresses.
        'localhost', 
         # Port number of the server.
        12345
    )

    async with server:
        await server.serve_forever()

# Start the `main` coroutine.
# This cannot be run from Jupyter notebook but must be run from a Python script.
asyncio.run(main())
```

### Echo server

A more conrete yet simple example for networking with `asyncio` is shown in the following.
The example consists of a server script `01_async_tcp_server.py` and a client script `01_async_tcp_client.py` which can both be found in the lecture folder. The server accepts TCP connections on the port 12345. The server reads data and sends it back to the client while changing all letters to upper case. The client script simply connects to the server, reads lines from the standard input and sends them to the server. At the same time it also receives data from the server and print it on the standard output.

The code is located in separate files because it cannot easily be run from Jupyter notebook. Therefore it has to be run from the terminal as follows:

```sh
# Start the server.
python3 01_async_tcp_server.py
```

In another terminal:
```sh
# Start the client (in another terminal).
python3 01_async_tcp_client.py --server 127.0.0.1 --port 12345
```

Study the two scripts!


# Exercises
Solve the exercises!